# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
movie_df=pd.read_csv('movies.txt',usecols=['movieId','title'],dtype={'movieId':'int32','title':'str'})
rating_df=pd.read_csv('ratings.txt',usecols=['userId','movieId','rating'],dtype={'userId':'int32','movieId':'int32','rating':'float'})

In [10]:
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [11]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [13]:
data=pd.merge(movie_df,rating_df,on='movieId')
data.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [29]:
movie_rating=data.dropna(axis=0,subset=['title'])
movie_ratingcount=movie_rating.groupby('title')['rating'].count().reset_index()
movie_ratingcount=movie_ratingcount.rename(columns={'rating':'totalRatingcount'})

In [30]:
movie_ratingcount.head()

,title,totalRatingcount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [32]:
rating_totalratingcount=movie_rating.merge(movie_ratingcount,how='left')
rating_totalratingcount.head()

,movieId,title,userId,rating,totalRatingcount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [33]:
threshold=50
rating_popular_movie=rating_totalratingcount.query('totalRatingcount >= @threshold')
rating_popular_movie.head()

,movieId,title,userId,rating,totalRatingcount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [34]:
rating_popular_movie.shape

(41362, 5)

In [37]:
#Pivot Table
movies_features_id=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movies_features_id.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [42]:
#Array Matrix
from scipy.sparse import csr_matrix
movie_features_df_matrix = csr_matrix(movies_features_id.values)

from sklearn.neighbors import NearestNeighbors

model=NearestNeighbors(metric='cosine',algorithm='brute')
model.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [48]:
query_index=np.random.choice(movies_features_id.shape[0])
print(query_index)

distances,indices=model.kneighbors(movies_features_id.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)


357


In [50]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movies_features_id.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movies_features_id.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Sin City (2005):

1: Kill Bill: Vol. 1 (2003), with distance of 0.33118897682485116:
2: V for Vendetta (2006), with distance of 0.3596383643417119:
3: Batman Begins (2005), with distance of 0.36903537843420253:
4: 300 (2007), with distance of 0.37684636251290593:
5: Kill Bill: Vol. 2 (2004), with distance of 0.3804421605428947:
